***

*Course:* [Math 535](https://people.math.wisc.edu/~roch/mmids/) - Mathematical Methods in Data Science (MMiDS)  
*Chapter:* 5-Spectral graph theory   
*Author:* [Sebastien Roch](https://people.math.wisc.edu/~roch/), Department of Mathematics, University of Wisconsin-Madison  
*Updated:* July 15, 2024   
*Copyright:* &copy; 2024 Sebastien Roch

***

In [ ]:
# You will need the files:
#     * mmids.py
# from https://github.com/MMiDS-textbook/MMiDS-textbook.github.io/tree/main/utils
#
# IF RUNNING ON GOOGLE COLAB (RECOMMENDED):
# "Upload to session storage" from the Files tab on the left
# Alternative instructions: https://colab.research.google.com/notebooks/io.ipynb

In [ ]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import torch
import mmids

$\newcommand{\bmu}{\boldsymbol{\mu}}$ 
$\newcommand{\bSigma}{\boldsymbol{\Sigma}}$
$\newcommand{\bfbeta}{\boldsymbol{\beta}}$ 
$\newcommand{\bflambda}{\boldsymbol{\lambda}}$
$\newcommand{\bgamma}{\boldsymbol{\gamma}}$ 
$\newcommand{\bsigma}{{\boldsymbol{\sigma}}}$
$\newcommand{\bpi}{\boldsymbol{\pi}}$
$\newcommand{\btheta}{{\boldsymbol{\theta}}}$ 
$\newcommand{\bphi}{\boldsymbol{\phi}}$ 
$\newcommand{\balpha}{\boldsymbol{\alpha}}$
$\newcommand{\blambda}{\boldsymbol{\lambda}}$
$\renewcommand{\P}{\mathbb{P}}$
$\newcommand{\E}{\mathbb{E}}$
$\newcommand{\indep}{\perp\!\!\!\perp} \newcommand{\bx}{\mathbf{x}}$ 
$\newcommand{\bp}{\mathbf{p}}$
$\renewcommand{\bx}{\mathbf{x}}$
$\newcommand{\bX}{\mathbf{X}}$ 
$\newcommand{\by}{\mathbf{y}}$ 
$\newcommand{\bY}{\mathbf{Y}}$ 
$\newcommand{\bz}{\mathbf{z}}$
$\newcommand{\bZ}{\mathbf{Z}}$ 
$\newcommand{\bw}{\mathbf{w}}$ 
$\newcommand{\bW}{\mathbf{W}}$
$\newcommand{\bv}{\mathbf{v}}$ 
$\newcommand{\bV}{\mathbf{V}}$
$\newcommand{\bfg}{\mathbf{g}}$ 
$\newcommand{\bfh}{\mathbf{h}}$ 
$\newcommand{\horz}{\rule[.5ex]{2.5ex}{0.5pt}}$
$\renewcommand{\S}{\mathcal{S}}$ 
$\newcommand{\X}{\mathcal{X}}$ 
$\newcommand{\var}{\mathrm{Var}}$ 
$\newcommand{\pa}{\mathrm{pa}}$
$\newcommand{\Z}{\mathcal{Z}}$ 
$\newcommand{\bh}{\mathbf{h}}$ 
$\newcommand{\bb}{\mathbf{b}}$ 
$\newcommand{\bc}{\mathbf{c}}$ 
$\newcommand{\cE}{\mathcal{E}}$ 
$\newcommand{\cP}{\mathcal{P}}$
$\newcommand{\bbeta}{\boldsymbol{\beta}}$
$\newcommand{\bLambda}{\boldsymbol{\Lambda}}$
$\newcommand{\cov}{\mathrm{Cov}}$
$\newcommand{\bfk}{\mathbf{k}}$
$\newcommand{\idx}[1]{}$
$\newcommand{\xdi}{}$

## Motivating example: uncovering social groups

In this chapter, we analyze datasets in the form of networks. As motivation, we first look at the [Karate Club dataset](https://en.wikipedia.org/wiki/Zachary%27s_karate_club). 

From [Wikipedia](https://en.wikipedia.org/wiki/Zachary%27s_karate_club):

> A social network of a karate club was studied by Wayne W. Zachary for a period of three years from 1970 to 1972. The network captures 34 members of a karate club, documenting links between pairs of members who interacted outside the club. During the study a conflict arose between the administrator "John A" and instructor "Mr. Hi" (pseudonyms), which led to the split of the club into two. Half of the members formed a new club around Mr. Hi; members from the other part found a new instructor or gave up karate. Based on collected data Zachary correctly assigned all but one member of the club to the groups they actually joined after the split.


We use the [`NetworkX`](https://networkx.org) package to load the data and visualize it (with help from Claude, who really likes the number `42`.). We will say more about it later in this chapter. In the meantime, there is a good tutorial [here](https://networkx.org/documentation/stable/tutorial.html).

In [ ]:
import networkx as nx
G = nx.karate_club_graph()

plt.figure(figsize=(6,6))
pos = nx.spring_layout(G, k=0.7, iterations=50, seed=42)
nx.draw_networkx(G, pos=pos, node_size=300, node_color='black', font_color='white')
plt.axis('off')
plt.show()

Our goal: 

> identify natural sub-groups in the network 

That is, we want to find groups of nodes that have many links between them, but relatively few with the other nodes. 

It will turn out that the eigenvectors of the Laplacian matrix, a matrix naturally associated to the graph, contain useful information about such communities.

**CHAT & LEARN** Ask your favority AI chatbot why it likes the number 42 so much. $\ddagger$

## Background: basic concepts in graph theory

**NUMERICAL CORNER:** In Python, the [`NetworkX`](https://networkx.org) package provides many functionalities for defining, modifying and plotting graphs. For instance, many standard graphs can be defined conveniently. The [`petersen_graph()`](https://networkx.org/documentation/stable/reference/generated/networkx.generators.small.petersen_graph.html) function defines the Petersen graph.

In [ ]:
G = nx.petersen_graph()

The graph can be plotted using the function [`networkx.draw_networkx()`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html). Recall that in NumPy array indices start at $0$. Consistently, NetworkX also names vertices starting at $0$. Note, however, that this conflicts with our mathematical conventions.

In [ ]:
nx.draw_networkx(G, node_color='black', font_color='white', node_size=200)
plt.axis('off')
plt.show()

Other standard graphs can be generated with special functions, e.g. complete graphs using [`complete_graph()`](https://networkx.org/documentation/stable/reference/generated/networkx.generators.classic.complete_graph.html#networkx.generators.classic.complete_graph). See [here](https://networkx.org/documentation/stable/reference/generators.html#module-networkx.generators.classic) for a complete list.

In [ ]:
G = nx.complete_graph(3)

nx.draw_networkx(G, node_color='black', font_color='white')
plt.axis('off')
plt.show()

See [here](https://networkx.org/documentation/stable/reference/functions.html) and [here](https://networkx.org/documentation/stable/reference/algorithms/index.html) for a list of functions to access various properties of a graph. Here are a few examples:

In [ ]:
G = nx.path_graph(10)

nx.draw_networkx(G, node_color='black', font_color='white')
plt.axis('off')
plt.show()

In [ ]:
G.number_of_nodes() # number of nodes

In [ ]:
G.number_of_edges() # number of edges

In [ ]:
G.has_node(7) # checks whether the graph has a particular vertex

In [ ]:
G.has_node(10)

In [ ]:
G.has_edge(0, 1) # checks whether the graph has a particular edge

In [ ]:
G.has_edge(0, 2)

In [ ]:
[n for n in G.neighbors(2)] # returns a list of neighbors of the specified vertex

In [ ]:
nx.is_connected(G) # checks whether the graph is connected

In [ ]:
[cc for cc in nx.connected_components(G)] # returns the connected components

In [ ]:
for e in G.edges():
    print(e)

Another way of specifying a graph is to start with an empty graph with a given number of vertices and then add edges one by one. The following command creates a graph with $4$ vertices and no edge (see [`empty_graph()`](https://networkx.org/documentation/stable/reference/generated/networkx.generators.classic.empty_graph.html#networkx.generators.classic.empty_graph)).

In [ ]:
G = nx.empty_graph(4)
G.add_edge(0, 1)
G.add_edge(2, 3)
G.add_edge(0, 3)
G.add_edge(3, 0)

nx.draw_networkx(G, node_color='black', font_color='white')
plt.axis('off')
plt.show()

$\unlhd$

**NUMERICAL CORNER:** The package `NetworkX` also supports digraphs.

In [ ]:
G = nx.DiGraph()
nx.add_star(G, [0, 1, 2, 3, 4])

nx.draw_networkx(G, node_color='black', font_color='white')
plt.axis('off')
plt.show()

Another way of specifying a digraph is to start with an empty graph with a given number of vertices and then add edges one by one (compare to the undirected case above).

In [ ]:
G = nx.DiGraph()
G.add_edge(0, 1)
G.add_edge(2, 3)
G.add_edge(0, 3)
G.add_edge(3, 0)
G.add_edge(1, 1)

nx.draw_networkx(G, node_color='black', font_color='white')
plt.axis('off')
plt.show()

Note that edges in both directions are depicted here with a double-arrow. Also the self-loop is hard to see. We can use 
[`networkx.draw_networkx_edges()`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx_edges.html#networkx.drawing.nx_pylab.draw_networkx_edges) (together with
[`networkx.draw_networkx_nodes()`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx_nodes.html#networkx.drawing.nx_pylab.draw_networkx_nodes) and
[`networkx.draw_networkx_labels()`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx_labels.html#networkx.drawing.nx_pylab.draw_networkx_labels))
to have more control over the drawing of the edges. 

In [ ]:
pos = nx.spring_layout(G, seed=42)
nx.draw_networkx_nodes(G, pos, node_color='black')
nx.draw_networkx_labels(G, pos, font_color='white')
nx.draw_networkx_edges(G, pos, connectionstyle="arc3,rad=0.3")
plt.axis('off')
plt.show()

$\unlhd$

**NUMERICAL CORNER:** Using `NetworkX`, the adjacency matrix of a graph can be obtained with [`adjacency_matrix()`](https://networkx.org/documentation/stable/reference/generated/networkx.linalg.graphmatrix.adjacency_matrix.html). By default, it returns a `SciPy` sparse matrix. Alternatively, one can get a regular array with [`toarray()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.toarray.html). 

In [ ]:
G = nx.complete_graph(4)
A = nx.adjacency_matrix(G)
print(A)

In [ ]:
A = nx.adjacency_matrix(G).toarray()
print(A)

The incidence matrix is obtained with [`incidence_matrix()`](https://networkx.org/documentation/stable/reference/generated/networkx.linalg.graphmatrix.incidence_matrix.html) -- again as a sparse array.

In [ ]:
B = nx.incidence_matrix(G)
print(B)

In [ ]:
B = nx.incidence_matrix(G).toarray()
print(B)

$\unlhd$

**NUMERICAL CORNER:** We revisit an earlier directed graph.

In [ ]:
G = nx.DiGraph()
G.add_edge(0, 1)
G.add_edge(2, 3)
G.add_edge(0, 3)
G.add_edge(3, 0)
G.add_edge(1,1)

We compute the adjacency and incidence matrices. For the incidence matrix, one must specify `oriented=True` for the oriented version.

In [ ]:
A = nx.adjacency_matrix(G).toarray()
print(A)

In [ ]:
B = nx.incidence_matrix(G, oriented=True).toarray()
print(B)

Revisiting an ealier undirected graph, we note that `incidence_matrix()` can also produce an arbitrary oriented incidence matrix by using the `oriented=True` option.

In [ ]:
G = nx.empty_graph(4)
G.add_edge(0, 1)
G.add_edge(2, 3)
G.add_edge(0, 3)
G.add_edge(3, 0)

B = nx.incidence_matrix(G, oriented=True).toarray()
print(B)

$\unlhd$

## Spectral properties of the Laplacian matrix

**NUMERICAL CORNER:** One use of the spectral decomposition of the Laplacian matrix is in graph drawing$\idx{graph drawing}\xdi$. We illustrate this next. Given a graph $G = (V, E)$, it is not clear a priori how to draw it in the plane since the only information available are adjacencies of vertices. One approach is just to position the vertices at random. The function [`networkx.draw()`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw.html) or [`networkx.draw_networkx()`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html#networkx.drawing.nx_pylab.draw_networkx)can take as input different [graph layout](https://networkx.org/documentation/stable/reference/drawing.html#module-networkx.drawing.layout) functions that return an $x$ and $y$-coordinate for each vertex.

We will test this on a grid graph. We use [`grid_2d_graph()`](https://networkx.org/documentation/stable/reference/generated/networkx.generators.lattice.grid_2d_graph.html) to construct such a graph.

In [ ]:
G = nx.grid_2d_graph(4,7)

One layout approach is to choose random locations for the nodes. Specifically, for every node, a position is generated by choosing each coordinate uniformly at random on the interval $[0,1]$.

In [ ]:
nx.draw_networkx(G, pos=nx.random_layout(G, seed=535), with_labels=False, 
                 node_size=50, node_color='black', width=0.5)
plt.axis('off')
plt.show()

Clearly, this is hard to read. 

Another approach is to map the vertices to two eigenvectors, similarly to what we did for dimensionality reduction. The eigenvector associated to $\mu_1$ is constant and therefore not useful for drawing. We try the next two. We use the Laplacian matrix. This is done using [`spectral_layout()`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spectral_layout.html#networkx.drawing.layout.spectral_layout).

In [ ]:
nx.draw_networkx(G, pos=nx.spectral_layout(G), with_labels=False, 
                 node_size=50, node_color='black', width=0.5)
plt.axis('off')
plt.show()

Interestingly, the outcome is provides a much more natural drawing of the graph, revealing its underlying structure as a grid. We will come back later to try to explain this, after we have developed further understanding of the spectral properties of the Laplacian matrix.

$\unlhd$

**NUMERICAL CORNER:** We construct a graph with two connected components and check the results above. We work directly with the adjacency matrix.

In [ ]:
A = np.array([[0, 1, 1, 0, 0], 
              [1, 0, 1, 0, 0], 
              [1, 1, 0, 0, 0], 
              [0, 0, 0, 0, 1], 
              [0, 0, 0, 1, 0]])
print(A)

Note the block structure.

The degrees can be obtained by summing the rows of the adjacency matrix.

In [ ]:
degrees = A.sum(axis=1)
print(degrees)

In [ ]:
D = np.diag(degrees)
print(D)

In [ ]:
L = D - A
print(L)

In [ ]:
print(LA.eigvals(L))

Observe that (up to numerical error) there are two $0$ eigenvalues and that the largest eigenvalue is greater or equal than the maximum degree plus one.

To compute the Laplacian matrix, one can also use the function `laplacian_matrix()`. For example, the Laplacian of the Petersen graph is the following:

In [ ]:
G = nx.petersen_graph()
L = nx.laplacian_matrix(G).toarray()
print(L)

In [ ]:
print(LA.eigvals(L))

$\unlhd$

**NUMERICAL CORNER:** This is perhaps easiest to see on a path graph. Recall that `NetworkX` numbers vertices $0,\ldots,n-1$. 

In [ ]:
G = nx.path_graph(10)

We plot the second Laplacian eigenvector (i.e., the eigenvector of the Laplacian matrix corresponding to the second smallest eigenvalue). We use [`numpy.argsort()`](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html) to find the index of the second smallest eigenvalue. Because indices start at `0`, we want entry `1` of the output of `np.argsort()`.

In [ ]:
L = nx.laplacian_matrix(G).toarray()
w, v = LA.eigh(L)
y2 = v[:,np.argsort(w)[1]]

plt.plot(y2, c='k')
plt.show()

$\unlhd$

## Application: graph partitioning via spectral clustering

**NUMERICAL CORNER:** **(A Random Tree)** We illustrate the definitions above on a tree, that is, a connected graph with no cycle. The function [`networkx.random_tree()`](https://networkx.org/documentation/stable/reference/generated/networkx.generators.trees.random_tree.html) can produce a random one. As before we use a `seed` for reproducibility. Again, we use $0,\ldots,n-1$ for the vertex set.

In [ ]:
G_tree = nx.random_tree(n=6, seed=111)

nx.draw_networkx(G_tree, pos=nx.circular_layout(G_tree), 
                 node_color='black', font_color='white')
plt.axis('off')
plt.show()

Suppose we take $S = \{0,1,2,3\}$. Then $S^c = \{4,5\}$ and 

$$
E(S,S^c)
= \{\{1,5\}, \{2,4\}\}.
$$

The cut ratio is then

$$
\phi(S)
= \frac{|E(S,S^c)|}{\min\{|S|,|S^c|\}}
= \frac{2}{2}
= 1.
$$

A better cut is given by $S = \{0,1,5\}$. In that case $S^c = \{2,3,4\}$, 

$$
E(S,S^c)= \{\{1,3\}\},
$$

and

$$
\phi(S)
= \frac{|E(S,S^c)|}{\min\{|S|,|S^c|\}}
= \frac{1}{3}.
$$

This is also equal to $\phi_G$. Indeed, in a connected graph with $n$ vertices, the numerator is at least $1$ and the denominator is at most $n/2$, which is achieved here.

$\unlhd$

**NUMERICAL CORNER:** We return to the random tree example above. We claimed that $\phi_G = 1/3$. The maximum degree is $\bar{\delta} = 3$. We now compute $\mu_2$. We first compute the Laplacian matrix.

In [ ]:
phi_G = 1/3
max_deg = 3

We now compute $\mu_2$. We first compute the Laplacian matrix.

In [ ]:
L_tree = nx.laplacian_matrix(G_tree).toarray()
print(L_tree)

In [ ]:
w, v = LA.eigh(L_tree) 
mu_2 = np.sort(w)[1]
print(mu_2)

We check *Cheeger's inequalities*. The left-hand side is:

In [ ]:
(phi_G ** 2) / (2 * max_deg)

The right-hand side is:

In [ ]:
2 * phi_G

$\unlhd$

We implement the graph cutting algorithm above.

We now implement this heuristic in Python. We first write an auxiliary function that takes as input an adjacency matrix, an ordering of the vertices and a value $k$. It returns the cut ratio for the first $k$ vertices in the order.

In [ ]:
def cut_ratio(A, order, k):
    
    n = A.shape[0]
    edge_boundary = 0
    for i in range(k+1):
        for j in range(k+1,n):
            edge_boundary += A[order[i],order[j]]
            
    denominator = np.minimum(k+1, n-k-1)
    
    return edge_boundary/denominator

Using the `cut_ratio` function, we first compute the Laplacian, find the second eigenvector and corresponding order of vertices. Then we compute the cut ratio for every $k$. Finally we output the cut (both $S_k$ and $S_k^c$) corresponding to the minimum, as a tuple of arrays.

In [ ]:
def spectral_cut2(A):
    n = A.shape[0]
    
    degrees = A.sum(axis=1)
    D = np.diag(degrees)
    L = D - A
    w, v = LA.eigh(L) 
    order = np.argsort(v[:,np.argsort(w)[1]])
    
    phi = np.zeros(n-1)
    for k in range(n-1):
        phi[k] = cut_ratio(A, order, k)
    imin = np.argmin(phi)
    
    return order[0:imin+1], order[imin+1:n]

Finally, to help visualize the output, we write a function coloring the vertices according to which side of the cut they are on.

In [ ]:
def viz_cut(G, s, pos, node_size=100, with_labels=False):
    n = G.number_of_nodes()
    assign = np.zeros(n)
    assign[s] = 1
    nx.draw(G, node_color=assign, pos=pos, with_labels=with_labels, 
            cmap='spring', node_size=node_size, font_color='k')
    plt.show()

**NUMERICAL CORNER:** We will illustrate this on the path graph.

In [ ]:
n = 10
G = nx.path_graph(n)

nx.draw_networkx(G, pos=nx.spectral_layout(G), 
                 node_color='black', font_color='white')
plt.axis('off')
plt.show()

We apply our spectral-based cutting algorihtm.

In [ ]:
A = nx.adjacency_matrix(G).toarray()
s, sc = spectral_cut2(A)
print(s)
print(sc)

In [ ]:
pos = nx.spectral_layout(G)
viz_cut(G, s, pos)

Let's try it on the grid graph. Can you guess what the cut will be?

In [ ]:
G = nx.grid_2d_graph(4,7)
A = nx.adjacency_matrix(G).toarray()
s, sc = spectral_cut2(A)
pos = nx.spectral_layout(G)
viz_cut(G, s, pos)

$\unlhd$

**NUMERICAL CORNER:** We return to the [Karate Club dataset](https://en.wikipedia.org/wiki/Zachary%27s_karate_club).

In [ ]:
G = nx.karate_club_graph()
n = G.number_of_nodes()
A = nx.adjacency_matrix(G).toarray()

We seek to find natural sub-communities. We use the spectral properties of the Laplacian. Specifically, we use our `spectral_cut2` and `viz_cut` functions to compute a good cut and vizualize it.

In [ ]:
s, sc = spectral_cut2(A)
print(s)
print(sc)

In [ ]:
plt.figure(figsize=(6,6))
pos = nx.spring_layout(G, k=0.7, iterations=50, seed=42)
viz_cut(G, s, pos, node_size=300, with_labels=True)

It is not trivial to assess the quality of the resulting cut. But this particular example has a known ground-truth community structure (which partly explains its widespread use). Quoting from [Wikipedia](https://en.wikipedia.org/wiki/Zachary%27s_karate_club):
> A social network of a karate club was studied by Wayne W. Zachary for a period of three years from 1970 to 1972. The network captures 34 members of a karate club, documenting links between pairs of members who interacted outside the club. During the study a conflict arose between the administrator "John A" and instructor "Mr. Hi" (pseudonyms), which led to the split of the club into two. Half of the members formed a new club around Mr. Hi; members from the other part found a new instructor or gave up karate. Based on collected data Zachary correctly assigned all but one member of the club to the groups they actually joined after the split.

This ground truth is the following. We use [`numpy.nonzero`](https://numpy.org/doc/stable/reference/generated/numpy.nonzero.html#numpy.nonzero) to convert it into a cut.

In [ ]:
truth = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 
                  0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
s_truth = np.nonzero(truth)
plt.figure(figsize=(6,6))
viz_cut(G, s_truth, pos, node_size=300, with_labels=True)

You can check that our cut perfectly matches the ground truth.

**CHAT & LEARN** Investigate alternative graph partitioning algorithms, such as the Kernighan-Lin algorithm. Ask your favorite AI chatbot to explain how it works and compare its performance to spectral clustering on this dataset. ([Open In Colab](https://colab.research.google.com/github/MMiDS-textbook/MMiDS-textbook.github.io/blob/main/just_the_code/roch_mmids_chap_specgraph_notebook.ipynb)) $\ddagger$

$\unlhd$

## Erdős-Rényi random graph and stochastic blockmodel

We implement the generation of an inhomogeneous ER graph using NetworkX. We first initialize a pseudorandom number generator `rng`. To determine whether an edge is present between `i` and `j`, we generate a uniform random variable `rng.random()` (see [`numpy.random.Generator.random`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.random.html)) and add the edge with `G.add_edge(i, j)` if the random variable is `< M[i, j]` -- an event which indeed occurs with the desired probability (check it!). 

In [ ]:
def inhomogeneous_er_random_graph(rng, n, M):

    G = nx.Graph()
    G.add_nodes_from(range(n))
    for i in range(n):
        for j in range(i + 1, n):
            if rng.random() < M[i, j]:
                G.add_edge(i, j)

    return G

**NUMERICAL CORNER:** Here is an example usage. We generate probabilities $m_{i,j}$ uniformly at random between $0$ and $1$.

In [ ]:
seed = 535
rng = np.random.default_rng(seed)

n = 20
M = rng.random([n, n])
M = (M + M.T) / 2 # ensures symmetry of M (why?)

G = inhomogeneous_er_random_graph(rng, n, M)

We draw the resulting graph.

In [ ]:
nx.draw_networkx(G, node_color='black', font_color='white')
plt.axis('off')
plt.show()

The following subroutine generates an ER graph. 

In [ ]:
def er_random_graph(rng, n, p):
    M = p * (np.ones((n, n)) - np.eye(n))
    return inhomogeneous_er_random_graph(rng, n, M)

To confirm our previous calculations, below is the implementation of a routine to estimate the edge density for an ER graph with a fixed parameter $p$. Recall that the edge density is defined as the number of edges present divided by the number of possible edges (i.e., the number of pairs of distinct vertices). The routine takes advantage of the *Law of Large Numbers* by generating a large number of sample graphs, computing their edge density, and then taking the mean. 

In [ ]:
def estimate_edge_density(rng, n, p, num_samples=100):

    total_edges = 0
    total_possible_edges = n * (n - 1) / 2
    
    for _ in range(num_samples):
        G = er_random_graph(rng, n, p)
        total_edges += G.number_of_edges()
    
    average_edges = total_edges / num_samples
    edge_density = average_edges / total_possible_edges
    return edge_density

**NUMERICAL CORNER:** On a small example, we indeed get that the edge density is roughly $p$.

In [ ]:
n = 10
p = 0.3
num_samples = 1000

estimated_density = estimate_edge_density(rng, n, p, num_samples)
print(f"Estimated edge density for an ER graph with n={n} and p={p}: {estimated_density}")

**TRY IT!** Modify the code above to estimate the density of triangles. ([Open In Colab](https://colab.research.google.com/github/MMiDS-textbook/MMiDS-textbook.github.io/blob/main/just_the_code/roch_mmids_chap_specgraph_notebook.ipynb)) $\ddagger$

$\unlhd$

When $n$, the number of vertices, is large, random graphs tend to exhibit large-scale emergent behavior. One classical example involves the probability of being connected in an ER graph. To illustrate, below is code to estimate that probability over a range of edge densities $p$ (with help from Claude and ChatGPT).

In [ ]:
def estimate_connected_probability(rng, n, p, num_samples=100):

    connected_count = 0
    
    for _ in range(num_samples):
        G = er_random_graph(rng, n, p)
        if nx.is_connected(G):
            connected_count += 1
    
    connected_probability = connected_count / num_samples
    return connected_probability

def plot_connected_probability(rng, n, p_values, num_samples=100):

    probabilities = []
    for p in p_values:
        prob = estimate_connected_probability(rng, n, p, num_samples)
        probabilities.append(prob)
    
    plt.figure(figsize=(6, 4))
    plt.plot(p_values, probabilities, marker='o', color='black')
    plt.xlabel('$p$'), plt.ylabel('Estimated probability of being connected')
    plt.show()

**NUMERICAL CORNER:** We run the code for `n` equal to `100`. What do you observe? 

In [ ]:
n = 100
p_values = np.linspace(0, 0.1, 50)
num_samples = 250
plot_connected_probability(rng, n, p_values, num_samples)

The probability of being connected starts out at $0$ when $p$ is small, which is not surprising since it implies that the graph has a relatively small number of edges. But then that probability increases -- rapidly -- to $1$ as $p$ crosses a threshold. This is referred to as the phase transition of the ER graph.

It can be shown rigorously that the transition occurs at roughly $p = \log n/n$. That is:

In [ ]:
np.log(n)/n

which is consistent with the plot.

**TRY IT!** Taking a larger `n` would produce a sharper transition. Try it for yourself. Also try drawing one random sample for increasing values of $p$ around the threshold. What do you observe? ([Open In Colab](https://colab.research.google.com/github/MMiDS-textbook/MMiDS-textbook.github.io/blob/main/just_the_code/roch_mmids_chap_specgraph_notebook.ipynb)) $\ddagger$

**TRY IT!** Many other properties exhibit such sharp threshold behavior. Modify the code to to estimate the probability that a clique of size 4 exists in the graph. ([Open In Colab](https://colab.research.google.com/github/MMiDS-textbook/MMiDS-textbook.github.io/blob/main/just_the_code/roch_mmids_chap_specgraph_notebook.ipynb)) $\ddagger$ 

$\unlhd$

We implement the SBM model. We use blocks numbered $0$ and $1$.

In [ ]:
def sbm_random_graph(rng, n, block_assignments, B):

    num_blocks = B.shape[0]
    Z = np.zeros((n, num_blocks))
    for i in range(n):
        Z[i, block_assignments[i]] = 1
    M = Z @ B @ Z.T
    
    return inhomogeneous_er_random_graph(rng, n, M)

**NUMERICAL CORNER:** Here is an example usage. We first pick a block assignment at random. Specifically, blocks are assigned randomly with [`numpy.random.Generator.choice`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.choice.html#numpy.random.Generator.choice). It produces two blocks by assigning each vertex with equal probability to either block, independently of all other choices.

In [ ]:
n = 50
block_assignments = rng.choice(2, n)  # randomly assign vertices to two blocks
B = np.array([[0.8, 0.1], [0.1, 0.8]])

G = sbm_random_graph(rng, n, block_assignments, B)

We draw the graph with colored nodes based on block assignments. The "good" cut is clearly visible in this layout.

In [ ]:
plt.figure(figsize=(6,6))
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color=block_assignments, cmap='rainbow',
        node_size=200, font_size=10, font_color='white')
plt.show()

$\unlhd$

We introduce a subroutine which assigns blocks at random as follows. Let $\beta_1, \beta_2 \in [0,1]$ with $\beta_1 + \beta_2 = 1$ be the probability that a vertex belongs respectively to block $1$ and $2$. We collect these probabilities in the following vector

$$
\bbeta = (\beta_1, \beta_2).
$$

We pick block $z(i) \in \{1,2\}$ for each vertex $1 \leq i \leq n$ according to the distribution $\bbeta$, independently of all other vertices $\neq i$.

In [ ]:
def generate_block_assignments(rng, n, beta):
    return rng.choice(len(beta), size=n, p=beta)

**NUMERICAL CORNER:** Here is an example usage.

In [ ]:
n = 50
beta = [0.33, 0.67]
B = np.array([[0.5, 0.03], [0.03, 0.4]])

block_assignments = generate_block_assignments(rng, n, beta)
G = sbm_random_graph(rng, n, block_assignments, B)

Observe that the blocks are more unbalanced this time.

In [ ]:
plt.figure(figsize=(6,6))
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color=block_assignments, cmap=plt.cm.rainbow,
        node_size=200, font_size=10, font_color='white')
plt.show()

To test our spectral partitioning algorithm, we run `spectral_cut2`, which indeed recovers the ground truth.

In [ ]:
A = nx.adjacency_matrix(G).toarray()
s, sc = mmids.spectral_cut2(A)

plt.figure(figsize=(6,6))
mmids.viz_cut(G, s, pos, node_size=200, with_labels=True)

$\unlhd$

The following code computes the fraction of incorrectly assigned vertices. Note that it considers *two* assignments corresponding to swapping the labels `0` and `1` which cannot be inferred. 

In [ ]:
def calculate_incorrect_fraction(block_assignments, inferred_s, inferred_sc):
    
    n = len(block_assignments)
    
    inferred_assignments = np.zeros(n)
    for i in inferred_s:
        inferred_assignments[i] = 0
    for i in inferred_sc:
        inferred_assignments[i] = 1
    
    incorrect_assignments_1 = np.sum(block_assignments != inferred_assignments)/n
    incorrect_assignments_2 = np.sum(block_assignments == inferred_assignments)/n

    return np.minimum(incorrect_assignments_1, incorrect_assignments_2)

**NUMERICAL CORNER:** We confirm on our previous example that the ground truth was perfectly recovered.

In [ ]:
fraction_incorrect = calculate_incorrect_fraction(block_assignments, s, sc)
print(f"Fraction of incorrectly assigned vertices: {fraction_incorrect}")

One expects that the ground truth is harder to recover if the probability of an edge between blocks is close to that within blocks, which makes the community structure more murky. To test this hypothesis, we modify our previous example by significantly increasing the inter-block probability.

In [ ]:
n = 100
beta = [0.55, 0.45]
B = np.array([[0.55, 0.25], [0.25, 0.45]])

block_assignments = generate_block_assignments(rng, n, beta)
G = sbm_random_graph(rng, n, block_assignments, B)

We run `spectral_cut2`. It recovers the ground truth only partially this time.

In [ ]:
A = nx.adjacency_matrix(G).toarray()
s, sc = mmids.spectral_cut2(A)
fraction_incorrect = calculate_incorrect_fraction(block_assignments, s, sc)
print(f"Fraction of incorrectly assigned vertices: {fraction_incorrect}")

$\unlhd$